In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import re

%matplotlib inline

In [ ]:
df = pd.read_csv('data/disaster_or_not/train.csv')
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df.shape

In [ ]:
df.drop('id', axis=1, inplace=True)
df.head()

In [ ]:
df['text'] = df.text.str.lower()

In [ ]:
df.head()

In [ ]:
df.keyword.isna()

In [ ]:
df.keyword.unique()

In [ ]:
df.loc[df['keyword'] == 'flooding', 'keyword', ] = 'flood'

In [ ]:
df.keyword.unique()

In [ ]:
df.loc[~df.keyword.isna()]

In [ ]:
df.loc[df.location.isna(), 'text'][0:5].values

In [ ]:
df.loc[df.location.isna(), 'location'][0:5].values

### Important note 
if we can find out city, state and country name from tweets then generted those location based on the the presence of it. If we don't find the locations then replace it with "unknown" word in location column

In [ ]:
df.loc[df.location.isna(), 'location',] = 0
df.loc[~df.location.isna(), 'location',] = 1

df.head()

In [ ]:
df.isnull().sum()

In [ ]:
def check_location(query):
    cities = pd.read_csv('data/cities.csv')
    states = pd.read_csv('data/states.csv')
    countries = pd.read_csv('data/countries.csv')
    
    for word in query.split():
        if word in cities:
            location = word
        elif word in states:
            lcaotion = word
        elif word in countries:
            location = word
        else:
            location = 'unknown'
            
        return location

df['location'] = df['text'].apply(check_location)

In [ ]:
df.head()

In [ ]:
def sep_word_symbol(query):
    words = []
    for w in query.split(' '):
        word_tokens = re.findall(r'(([a-z0-9]+)([.#\"\'?;:])([a-z]+)?)', w)
        if len(word_tokens):
            w = ' '.join([tok for tok in word_tokens[0][1:]])
            words.append(w)
        else:
            words.append(w)
    
    return ' '.join([d for d in words])

In [ ]:
df['text'] = df['text'].apply(sep_word_symbol)
df.head()

In [ ]:
df.text.str.replace(',', '')

In [ ]:
freq = pd.Series(' '.join(df['text']).split()).value_counts()
freq

In [ ]:
df['text'] = df['text'].apply(lambda x: " ".join(w for w in x.split() if freq.get(w) > 5))

In [ ]:
freq = pd.Series(' '.join(df['text']).split()).value_counts()
freq

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(df['text'], df['target'], 
                                                      test_size = 0.2, random_state = 4)

In [ ]:
cv = CountVectorizer(stop_words='english', ngram_range=(1, 2))
X_traincv = cv.fit_transform(X_train)
X_validcv = cv.transform(X_valid)

In [ ]:
cv.get_feature_names()

In [ ]:
from sklearn.naive_bayes import MultinomialNB
naive_bayes = MultinomialNB()
naive_bayes.fit(X_traincv, y_train)

In [ ]:
# Testing accuracy
naive_bayes.score(X_validcv, y_valid)

In [ ]:
# Traiining accuracy
naive_bayes.score(X_traincv, y_train)